# Littlefinger

   -*Slyly telling you where your money goes*

## Table of Contents

TODO

Reports:
For both just US and UK and all transactions in GBP as it cost me:

Master categories, table, graph:
Income per month
Expenses per month


## Boilerplate

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from dateutil.parser import parse
import seaborn as sns
import matplotlib.pyplot as plt
from pandas.tseries.offsets import *

%matplotlib inline
%load_ext autoreload
%autoreload 2
pd.set_option('float_format', '{:.2f}'.format)
plt.style.use('seaborn-colorblind')
cm = sns.light_palette("green", as_cmap=True)

## Import data and set variables

In [2]:
excel = pd.ExcelFile("C:\\Users\\aowd\OneDrive - Chevron\\Special Projects\\littlefinger\\Money1.2.xlsx")
uk = excel.parse('UK')
us = excel.parse('US')
funds = excel.parse('Investments')
prices = excel.parse('Funds')
forex = excel.parse('Forex')

now = datetime.now()
weekago = now - timedelta(7)
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
rows = ['Car', 'Entertainment', 'Fees', 'Food', 'Gift', 'Health', 'Home', 'Kids', 'NBC', 'Pets', 'Shopping', 'Travel']

## Preprocess Data

In [3]:
def tidyxacts(df):
    """
    Fix formats of columns for transactions in dataframe df

    Parameters
    ----------
    df : Dataframe

    Returns
    -------
    Date formatted as Date
    Amounts as numeric
    Adds columns for year, month, quarter, week number
    Sets date as index
    """

    df['Date'] = pd.to_datetime(df['Date'])
    df['Inflow'] = pd.to_numeric(df['Inflow'])
    df['Outflow'] = pd.to_numeric(df['Outflow'])
    df['Net'] = pd.to_numeric(df['Net'])
    # df['Master Category'] = df['Master Category'].astype("category")
    # df['Sub Category'] = df['Sub Category'].astype("category")
    # df['Year'], df['Month'] = df['Date'].dt.year, df['Date'].dt.strftime('%b')
    df['Year'], df['Month'] = df['Date'].dt.year, df['Date'].dt.month
    df['Quarter'], df['Week'] = df['Date'].dt.quarter, df['Date'].dt.week
    df.set_index(['Date'], inplace=True)
   
def xact_type(df):
    """
    Adds transfer, expense, income category column to df  
    """
    def categorise(row):
        if row['Master Category'] == "Transfer":
            return 'Transfer'
        if row['Master Category'] == "Income":
            return 'Income'
        return 'Expense'

    df['Type'] = df.apply(lambda row: categorise(row), axis=1)

    
tidyxacts(uk)
tidyxacts(us)
xact_type(uk)
xact_type(us)

### Create all transaction table in GBP

In [4]:
forex.set_index(['Date'], inplace=True) # Set date to be index
forex = forex['Exchange Rate'] # Only keep exchange column
usex =  us.join(other=forex, how='outer')
usex['Exchange Rate'].fillna(method='ffill', inplace=True)
usex['Net GBP'] = usex['Net'] / usex['Exchange Rate']
usgbp = usex.drop(['Net', 'Exchange Rate'], axis=1)
usgbp.rename(columns={'Net GBP':'Net'}, inplace=True)
allgbp = pd.concat([uk, usgbp], axis=0)

## Results

### All Expenses in GBP

In [5]:
### Summary Table
summary = pd.pivot_table(allgbp['2017'].query('Type == ["Expense"]'), index=['Master Category'], columns=['Month'], values=['Net'], aggfunc=np.sum, fill_value=0)
summary.columns = months[:now.month] # Renames to months based off current month
summary = summary * -1 # We're just looking at expenses
summary['Total'] = summary.sum(axis=1) # adds total to categories
summary.loc['Total']= summary.sum() # adds total row at bottom
summary.style.format("£{:,.2f}")


In [6]:
# Max expense month per category
summary.style.highlight_max(color='pink', axis=1).format("£{:,.2f}")

In [7]:
# Heatmap
cm = sns.light_palette("green", as_cmap=True)
hm_summary = summary.style.background_gradient(cmap=cm, subset=(summary.index[:-1], summary.columns[:])).format("£{:,.2f}").set_caption('Monthly Summary')
hm_summary

In [8]:
# Expense per month Heatmap
# Ignoring housing and total rows - tend to mask the rest
summary.style.background_gradient(cmap=cm, subset=pd.IndexSlice[rows,summary.columns[:-1]], axis=1).format("£{:,.2f}")